In [41]:
import gc

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [195]:
dataset_path = 'external_datasets/vehicles.csv'
df = pd.read_csv(dataset_path)#, usecols=['franchise_make', 'model_name', 'year', 'price'])
print(df.shape)

(426880, 26)


In [196]:
df = df[['manufacturer', 'model', 'year', 'price']].dropna(axis=0)
df.shape

(404020, 4)

In [200]:
grouped = df[['manufacturer', 'model', 'year', 'price']]\
    .rename({'manufacturer':'make'}, axis=1)\
    .groupby(['make', 'model', 'year'], as_index=False).agg(['mean', 'count'])
grouped.columns = ['make', 'model', 'year', 'price', 'count']
grouped.to_csv('grouped_car_prices_vehicles.csv', index=False)
grouped.shape

(56752, 5)

In [205]:
filtered = grouped[grouped['year'] >= 2012]
filtered['make'] = filtered['make'].str.capitalize()
filtered['model'] = filtered['model'].str.capitalize()
filtered.shape

C:\Users\26823\AppData\Local\Temp\ipykernel_12184\1703322462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['make'] = filtered['make'].str.capitalize()
C:\Users\26823\AppData\Local\Temp\ipykernel_12184\1703322462.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['model'] = filtered['model'].str.capitalize()


(27599, 5)

In [310]:
dataset_path = 'external_datasets/used_cars_3M_US.csv'
df = pd.read_csv(dataset_path, usecols=['make_name', 'model_name', 'year', 'price', 'salvage'])
print(df.shape)

(3000040, 5)


In [303]:
# pd.options.display.max_columns = 500

In [313]:
grouped = df[['make_name', 'model_name', 'year', 'price']]\
    .rename({'make_name':'make', 'model_name':'model'}, axis=1)\
    .groupby(['make', 'model', 'year'], as_index=False).agg(['mean', 'count'])
grouped.to_csv('grouped_car_prices_3MUS.csv', index=False)
grouped.shape

(10423, 5)

In [312]:
filtered = grouped[grouped['year'] >= 2012]
filtered.columns = ['make', 'model', 'year', 'price', 'count']
filtered.shape

(3659, 5)

In [278]:
# df.drop(df.index, inplace=True)
# del df
# gc.collect()

In [315]:
data_path = 'scraped/dataset_turo-scraper_Central_Valley.json'
scraped_df = pd.read_json(data_path)
scraped_df.shape

(81, 36)

In [325]:
veh_df    = pd.json_normalize(scraped_df['vehicle'])
locations = pd.json_normalize(scraped_df['location'])
rates     = pd.json_normalize(scraped_df['rate'])
owner     = pd.json_normalize(scraped_df['owner'])
big = pd.concat([scraped_df, veh_df, locations, rates, owner], axis=1)\
    .drop(['rate', 'vehicle', 'location', 'owner'], axis=1)
big = big[(big['year'] >= 2012) & (big['year'] < 2022)]
print(big.shape)
del veh_df, locations, rates, owner

(49, 122)


In [326]:
merged = pd.merge(big, filtered, on=['make', 'model', 'year'], how='left', indicator='Exist')
merged['Exist'].value_counts()

Exist
both          33
left_only     16
right_only     0
Name: count, dtype: int64

In [333]:
# merged[merged['Exist'] != 'both'][['make', 'model', 'year']]

In [283]:
# {
#     'Titan XD': 'Titan'
# },\
# {
#     'MAZDA3': 'Mazda3'
#     , 'CT *':'CT'
#     , 'XJ *': 'XJ'
#     , 'GS *': 'GS'
#     , 'Transit *': 'Transit'
#     , 'RX *': 'RX'
#     , 'RC *': 'RC'
# }

In [334]:
# filtered[(filtered['make'] == 'Porsche') & (filtered['model'].str.contains('Macan'))]

In [336]:
merged = merged[merged['Exist'] == 'both']
merged.to_csv('merged_prices_Central_Valley.csv', index=False)
merged.head()

,airportLocations,badges,basicCarDetails,color,currentVehicleProtection,dateRangeRate,description,extras,freeDeliveryPeriodDescription,frequentlyBooked,guestProtectionDisclaimer,guidelines,highValueVehicle,images,inNeedOfAirportMigration,inspectionDetails,instantBookLocationPreferences,listingDeleted,listingEnabled,minimumAgeInYearsToRent,numberOfFaqs,numberOfFavorites,numberOfRentals,numberOfReviews,parkingDetails,ratings,tieredOwnerProtectionEnabled,tripCount,turoGo,turoGoExplanation,vehicleDeliveryLocations,vehicleValueType,automaticTransmission,id,listingCreatedTime,make,marketAreaId,marketCountry,model,name,trim,type,url,vin,year,image.darkModeImageUrl,image.id,image.originalImageUrl,image.placeholder,image.resizableUrlTemplate,image.thumbnails.170x125,image.thumbnails.170x102,image.thumbnails.620x372,image.thumbnails.574x343,image.thumbnails.50x30,image.thumbnails.100x60,image.verified,registration.insuranceCardUrl,registration.licensePlate,registration.regionRequired,registration.state,marketCurrency.currencyCode,marketCurrency.decimalPlaces,marketCurrency.defaultFractionDigits,marketCurrency.symbol,address,addressLines,city,country,id,latitude,locationSource,longitude,state,timeZone,precision.accuracy,precision.level,airportDeliveryFee,airportDeliveryLocationsAndFees,averageDailyPrice,customDeliveryDistance,customDeliveryFee,daily,dailyMileage,homeLocationFee,monthlyDiscountPercentage,monthlyMileage,promotionResponse,regularAirportDeliveryFee,regularCustomDeliveryFee,weeklyDiscountPercentage,weeklyMileage,averageDailyPriceWithCurrency.amount,averageDailyPriceWithCurrency.currencyCode,dailyDistance.scalar,dailyDistance.unit,dailyDistance.unlimited,excessFeePerDistance.amount,excessFeePerDistance.currencyCode,monthlyDistance.scalar,monthlyDistance.unit,monthlyDistance.unlimited,weeklyDistance.scalar,weeklyDistance.unit,weeklyDistance.unlimited,allStarHost,firstName,id,lastName,name,proHost,url,image.darkModeImageUrl,image.id,image.originalImageUrl,image.placeholder,image.resizableUrlTemplate,image.thumbnails.84x84,image.thumbnails.300x300,image.thumbnails.225x225,image.thumbnails.32x32,image.verified,price,count,Exist
0,[],"[{'id': 26, 'label': 'Android Auto', 'value': ...","{'averageFuelEconomy': 32.5, 'averageFuelEcono...",None,"{'companyInfo': None, 'deliveryOwnerFraction':...","{'appliedDiscountUnit': 'DAILY', 'days': 3, 'd...",LOOKING FOR A SMOOTH RIDE??\nYOU FOUND IT!\n\n...,{'extras': []},None,False,All trips are covered under a third-party liab...,None,False,"[{'darkModeImageUrl': None, 'id': 16000909, 'o...",False,NaN,"{'airportLocationEnabled': False, 'customLocat...",False,True,NaN,0,21,19,19,None,{'histogram': {'buckets': [{'averageRating': 5...,True,19,False,None,[],"{'explanation': {'details': [], 'subtitle': No...",True,2407767,1698341376000,Chevrolet,None,US,Malibu,Corinthia's Chevrolet,LS,CAR,https://turo.com/us/en/car-rental/united-state...,None,2021,None,16000909,https://images.turo.com/media/vehicle/images/c...,False,https://images.turo.com/media/vehicle/images/c...,https://images.turo.com/media/vehicle/images/c...,https://images.turo.com/media/vehicle/images/c...,https://images.turo.com/media/vehicle/images/c...,https://images.turo.com/media/vehicle/images/c...,https://images.turo.com/media/vehicle/images/c...,https://images.turo.com/media/vehicle/images/c...,False,https://help.turo.com/SJJELVe4c,None,True,CA,USD,2,2,USD,"Merced, CA 95348","[Merced, CA]",Merced,US,24508461,37.320745,GOOGLE,-120.482572,CA,America/Los_Angeles,0.093206,APPROXIMATE,None,[],47.50,Up to 50 miles,$65,47.50,250.0,Free,20,7500.0,None,None,$65,10,1750.0,47.50,USD,250.0,MI,False,0.19,USD,7500.0,MI,False,1750.0,MI,False,True,Corinthia,26837729,H.,Corinthia H.,False,https://turo.com/us/en/drivers/26837729,None,None,https://images.turo.com/media/driver/-s6JbPJ8R...,False,https://images.turo.com/media/driver/-s6JbPJ8R...,https://images.turo.com/media/driver/-s6JbPJ8R...,https://images.turo.com/media/driver/-s6JbPJ8R...,https://imag

### Prices Scraper

In [48]:
# big['make_and_model'] = big[['make', 'model']].apply(lambda x: x['make'].lower() + '-' + x['model'].lower().replace(' ', '-'), axis=1)
# all_makemodels = big['make_and_model'].unique()
# len(all_makemodels)

267

In [49]:
# links = ['https://www.iseecars.com/car/' + all_makemodels[i] for i in range(len(all_makemodels))]
# driver = webdriver.Chrome()
# driver.get(links[0])
# section_name = "Used Car Pricing"
# table = driver.find_element(By.XPATH, f"//section[@name='{section_name}']")
# # table = driver.find_element(By.XPATH, table_xpath)
# # cell_xpath = f"//table/tbody/tr[{2 + 1}]/td[{3 + 1}]"
# # cell = table.find_element(By.XPATH, cell_xpath)
# # print(cell.text)
# table_text = table.text
# print(table_text)
# driver.quit()